In [9]:
import pandas as pd

import numpy as np
import os

import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

In [18]:
import pandas as pd

# 데이터 파일 읽기
df = pd.read_csv('Data/기타/서울시_상권분석서비스(점포-행정동)_2022년.csv', encoding='euc-kr')

# 편의점 데이터 필터링 (서비스_업종_코드_명이 '편의점'인 행 선택)
convenience_stores = df[df['서비스_업종_코드_명'] == '편의점']

# 행정동별 편의점의 유사업종 점포수 합계 계산
district_similar_stores = convenience_stores.groupby('행정동_코드_명')['유사_업종_점포_수'].sum().reset_index()

# 열 이름 변경
district_similar_stores.columns = ['행정동', '편의점_유사업종_점포수']

# 결과를 유사업종 점포수 기준으로 내림차순 정렬
district_similar_stores = district_similar_stores.sort_values('편의점_유사업종_점포수', ascending=False)

# 상위 20개 행정동만 선택 (선택사항)
top_20_districts = district_similar_stores.head(20)

# 하위 20개 행정동만 선택 (선택사항)
tail_20_districts = district_similar_stores.tail(20)
# 결과 출력
print("행정동별 편의점 유사업종 점포수 (상위 20개):")
print(top_20_districts.to_string(index=False))

print(tail_20_districts.to_string(index=False))


행정동별 편의점 유사업종 점포수 (상위 20개):
        행정동  편의점_유사업종_점포수
       역삼1동           735
        가산동           555
        서교동           549
종로1·2·3·4가동           506
        여의동           458
         명동           406
       가양1동           351
        신촌동           333
       영등포동           320
       문정2동           317
       논현1동           310
       서초3동           304
        신사동           302
       구로3동           295
       논현2동           290
        청담동           274
       장안1동           255
        회현동           255
       삼성1동           249
        화양동           247
  행정동  편의점_유사업종_점포수
  가회동            29
 홍제2동            29
 잠실4동            28
 암사3동            28
 돈암2동            28
상계10동            25
  청림동            24
  삼청동            23
  수색동            22
 상계8동            21
  청구동            20
 돈암1동            20
  난향동            18
 창신3동            16
  응봉동            14
 잠실7동            14
 창신2동            13
 둔촌1동            12
 시흥2동            12
 개포1동             8


In [17]:
df.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
0,20221,11110515,청운효자동,CS100001,한식음식점,92,93,2,2,3,3,1
1,20221,11110515,청운효자동,CS100002,중식음식점,3,3,0,0,0,0,0
2,20221,11110515,청운효자동,CS100003,일식음식점,12,12,8,1,0,0,0
3,20221,11110515,청운효자동,CS100004,양식음식점,46,46,0,0,2,1,0
4,20221,11110515,청운효자동,CS100005,제과점,22,23,13,3,0,0,1


In [19]:
# 2. 편의점 데이터 필터링
print("\n2. 편의점 데이터 필터링")
convenience_stores = df[df['서비스_업종_코드_명'] == '편의점']
print(convenience_stores.head())
print("\n편의점 데이터 수:", len(convenience_stores))


2. 편의점 데이터 필터링
     기준_년분기_코드    행정동_코드 행정동_코드_명 서비스_업종_코드 서비스_업종_코드_명  점포_수  유사_업종_점포_수  \
45       20221  11110515    청운효자동  CS300002         편의점     2          14   
135      20221  11110530      사직동  CS300002         편의점     1          33   
212      20221  11110540      삼청동  CS300002         편의점     0           6   
281      20221  11110550      부암동  CS300002         편의점     0          11   
353      20221  11110560      평창동  CS300002         편의점     1          17   

     개업_율  개업_점포_수  폐업_률  폐업_점포_수  프랜차이즈_점포_수  
45      0        0     0        0          12  
135     0        0     0        0          32  
212     0        0     0        0           6  
281     9        1     9        1          11  
353     6        1     0        0          16  

편의점 데이터 수: 1700


In [22]:
# 3. 유사 업종 점포수 계산 (점포수 + 프랜차이즈 점포수)
print("\n3. 유사 업종 점포수 계산")
convenience_stores['유사_업종_점포수_계산'] = convenience_stores['점포_수'] + convenience_stores['프랜차이즈_점포_수']
print(convenience_stores[['행정동_코드_명', '점포_수', '프랜차이즈_점포_수', '유사_업종_점포_수', '유사_업종_점포수_계산']].head())


3. 유사 업종 점포수 계산
    행정동_코드_명  점포_수  프랜차이즈_점포_수  유사_업종_점포_수  유사_업종_점포수_계산
45     청운효자동     2          12          14            14
135      사직동     1          32          33            33
212      삼청동     0           6           6             6
281      부암동     0          11          11            11
353      평창동     1          16          17            17


/var/folders/5r/fgj26ncd2y9_f_q31f04nxvc0000gp/T/ipykernel_2959/1158785743.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  convenience_stores['유사_업종_점포수_계산'] = convenience_stores['점포_수'] + convenience_stores['프랜차이즈_점포_수']


In [23]:
# 4. 행정동별 편의점의 유사업종 점포수 합계 계산
print("\n4. 행정동별 편의점의 유사업종 점포수 합계 계산")
district_similar_stores = convenience_stores.groupby('행정동_코드_명')['유사_업종_점포수_계산'].sum().reset_index()
print(district_similar_stores.head())


4. 행정동별 편의점의 유사업종 점포수 합계 계산
  행정동_코드_명  유사_업종_점포수_계산
0     가락1동            44
1     가락2동            88
2     가락본동           190
3     가리봉동            51
4      가산동           555


In [24]:

district_similar_stores.columns = ['행정동', '편의점_유사업종_점포수']


In [25]:
# 6. 결과를 유사업종 점포수 기준으로 내림차순 정렬
district_similar_stores = district_similar_stores.sort_values('편의점_유사업종_점포수', ascending=False)


In [27]:

# 7. 상위 20개 행정동 선택
top_20_districts = district_similar_stores.head(20)

print("\n8. 최종 결과 (상위 20개 행정동):")
print(top_20_districts.to_string(index=False))



8. 최종 결과 (상위 20개 행정동):
        행정동  편의점_유사업종_점포수
       역삼1동           735
        가산동           555
        서교동           549
종로1·2·3·4가동           506
        여의동           458
         명동           406
       가양1동           351
        신촌동           333
       영등포동           320
       문정2동           317
       논현1동           310
       서초3동           304
        신사동           302
       구로3동           295
       논현2동           290
        청담동           274
       장안1동           255
        회현동           255
       삼성1동           249
        화양동           247


In [28]:

# 하위 20개 행정동만 선택 (선택사항)
tail_20_districts = district_similar_stores.tail(20)
# 결과 출력

print(tail_20_districts.to_string(index=False))

  행정동  편의점_유사업종_점포수
  가회동            29
 홍제2동            29
 잠실4동            28
 암사3동            28
 돈암2동            28
상계10동            25
  청림동            24
  삼청동            23
  수색동            22
 상계8동            21
  청구동            20
 돈암1동            20
  난향동            18
 창신3동            16
  응봉동            14
 잠실7동            14
 창신2동            13
 둔촌1동            12
 시흥2동            12
 개포1동             8


In [37]:

# 편의점 데이터 필터링
convenience_stores = df[df['서비스_업종_코드_명'] == '편의점']
print("\n편의점 데이터 필터링 완료. 편의점 데이터 shape:", convenience_stores.shape)

# 유사 업종 점포수 계산 (점포수 + 프랜차이즈 점포수)
convenience_stores['유사_업종_점포수'] = convenience_stores['점포_수'] + convenience_stores['프랜차이즈_점포_수']
print("\n유사 업종 점포수 계산 완료. 샘플 데이터:")
print(convenience_stores[['행정동_코드_명', '점포_수', '프랜차이즈_점포_수', '유사_업종_점포수']].head())

# 행정동별, 분기별 편의점의 유사업종 점포수 합계 계산
district_quarterly_stores = convenience_stores.groupby(['행정동_코드_명', '기준_년분기_코드'])['유사_업종_점포수'].sum().reset_index()
print("\n행정동별, 분기별 유사업종 점포수 계산 완료. 샘플 데이터:")
print(district_quarterly_stores.head())

# 연간 평균 계산
district_yearly_avg = district_quarterly_stores.groupby('행정동_코드_명')['유사_업종_점포수'].mean().reset_index()
district_yearly_avg.columns = ['행정동', '편의점_유사업종_평균_점포수']
print("\n연간 평균 계산 완료. 샘플 데이터:")
print(district_yearly_avg.head())

# 결과를 평균 점포수 기준으로 내림차순 정렬
district_yearly_avg = district_yearly_avg.sort_values('편의점_유사업종_평균_점포수', ascending=False)

# 상위 20개 행정동 선택
top_20_districts = district_yearly_avg.head(20)

# 결과 출력
print("\n행정동별 편의점 유사업종 평균 점포수 (상위 20개):")
print(top_20_districts.to_string(index=False))



편의점 데이터 필터링 완료. 편의점 데이터 shape: (1700, 12)

유사 업종 점포수 계산 완료. 샘플 데이터:
    행정동_코드_명  점포_수  프랜차이즈_점포_수  유사_업종_점포수
45     청운효자동     2          12         14
135      사직동     1          32         33
212      삼청동     0           6          6
281      부암동     0          11         11
353      평창동     1          16         17

행정동별, 분기별 유사업종 점포수 계산 완료. 샘플 데이터:
  행정동_코드_명  기준_년분기_코드  유사_업종_점포수
0     가락1동      20221         11
1     가락1동      20222         11
2     가락1동      20223         11
3     가락1동      20224         11
4     가락2동      20221         22

연간 평균 계산 완료. 샘플 데이터:
    행정동  편의점_유사업종_평균_점포수
0  가락1동            11.00
1  가락2동            22.00
2  가락본동            47.50
3  가리봉동            12.75
4   가산동           138.75

행정동별 편의점 유사업종 평균 점포수 (상위 20개):
        행정동  편의점_유사업종_평균_점포수
       역삼1동           183.75
        가산동           138.75
        서교동           137.25
종로1·2·3·4가동           126.50
        여의동           114.50
         명동           101.50
       가양1동            87.75
        신촌

/var/folders/5r/fgj26ncd2y9_f_q31f04nxvc0000gp/T/ipykernel_2959/2356233990.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  convenience_stores['유사_업종_점포수'] = convenience_stores['점포_수'] + convenience_stores['프랜차이즈_점포_수']


In [39]:
import pandas as pd

# 데이터 파일 읽기
df = pd.read_csv('Data/기타/서울시_상권분석서비스(점포-행정동)_2022년.csv', encoding='cp949')
print("데이터 로드 완료. 데이터 shape:", df.shape)
print("컬럼 목록:", df.columns.tolist())

# 편의점 데이터 필터링
convenience_stores = df[df['서비스_업종_코드_명'] == '편의점']
print("\n편의점 데이터 필터링 완료. 편의점 데이터 shape:", convenience_stores.shape)

# 유사 업종 점포수 계산 (점포수 + 프랜차이즈 점포수)
convenience_stores['유사_업종_점포수'] = convenience_stores['점포_수'] + convenience_stores['프랜차이즈_점포_수']
print("\n유사 업종 점포수 계산 완료. 샘플 데이터:")
print(convenience_stores[['행정동_코드_명', '점포_수', '프랜차이즈_점포_수', '유사_업종_점포수']].head())

# 행정동별, 분기별 편의점의 유사업종 점포수 합계 계산
district_quarterly_stores = convenience_stores.groupby(['행정동_코드_명', '기준_년분기_코드'])['유사_업종_점포수'].sum().reset_index()
print("\n행정동별, 분기별 유사업종 점포수 계산 완료. 샘플 데이터:")
print(district_quarterly_stores.head())

# 연간 평균 계산
district_yearly_avg = district_quarterly_stores.groupby('행정동_코드_명')['유사_업종_점포수'].mean().reset_index()
district_yearly_avg.columns = ['행정동', '편의점_유사업종_평균_점포수']
print("\n연간 평균 계산 완료. 샘플 데이터:")
print(district_yearly_avg.head())

# 결과를 평균 점포수 기준으로 내림차순 정렬
district_yearly_avg = district_yearly_avg.sort_values('편의점_유사업종_평균_점포수', ascending=False)

# 상위 20개 행정동 선택
top_20_districts = district_yearly_avg.head(20)

# 결과 출력
print("\n행정동별 편의점 유사업종 평균 점포수 (상위 20개):")
print(top_20_districts.to_string(index=False))



데이터 로드 완료. 데이터 shape: (141334, 12)
컬럼 목록: ['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드', '서비스_업종_코드_명', '점포_수', '유사_업종_점포_수', '개업_율', '개업_점포_수', '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수']

편의점 데이터 필터링 완료. 편의점 데이터 shape: (1700, 12)

유사 업종 점포수 계산 완료. 샘플 데이터:
    행정동_코드_명  점포_수  프랜차이즈_점포_수  유사_업종_점포수
45     청운효자동     2          12         14
135      사직동     1          32         33
212      삼청동     0           6          6
281      부암동     0          11         11
353      평창동     1          16         17

행정동별, 분기별 유사업종 점포수 계산 완료. 샘플 데이터:
  행정동_코드_명  기준_년분기_코드  유사_업종_점포수
0     가락1동      20221         11
1     가락1동      20222         11
2     가락1동      20223         11
3     가락1동      20224         11
4     가락2동      20221         22

연간 평균 계산 완료. 샘플 데이터:
    행정동  편의점_유사업종_평균_점포수
0  가락1동            11.00
1  가락2동            22.00
2  가락본동            47.50
3  가리봉동            12.75
4   가산동           138.75

행정동별 편의점 유사업종 평균 점포수 (상위 20개):
        행정동  편의점_유사업종_평균_점포수
       역삼1동           183.75
    

/var/folders/5r/fgj26ncd2y9_f_q31f04nxvc0000gp/T/ipykernel_2959/4130714752.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  convenience_stores['유사_업종_점포수'] = convenience_stores['점포_수'] + convenience_stores['프랜차이즈_점포_수']
